In [ ]:
!pip install transformers torch pdfplumber python-docx python-pptx openpyxl pytesseract pillow pandas python-magic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 18.4 MB/s eta 0:00:00


In [ ]:
!apt-get update -qq && apt-get install -y -qq tesseract-ocr libtesseract-dev



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Selecting previously unselected package libleptonica-dev.
Preparing to unpack .../libleptonica-dev_1.82.0-3build1_amd64.deb ...
Unpacking libleptonica-dev (1.82.0-3build1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack .../libtesseract-dev_4.1.1-2.1build1_amd64.deb ...
Unpacking libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Setting up libleptonica-dev (1.82.0-3build1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Processing triggers for ma

In [ ]:

import os
import re
import io
import mimetypes
import pdfplumber
import docx
from pptx import Presentation
import openpyxl
from PIL import Image
import pytesseract
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"


In [ ]:
import os
import pdfplumber
import docx
from pptx import Presentation
import openpyxl
from PIL import Image
import pytesseract
import mimetypes
import re
def detect_file_type(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in [".pdf"]: return "pdf"
    if ext in [".docx", ".doc"]: return "docx"
    if ext in [".pptx"]: return "pptx"
    if ext in [".xlsx", ".xls", ".csv"]: return "xlsx"
    if ext in [".png", ".jpg", ".jpeg", ".tiff", ".bmp"]: return "image"
    return "unknown"
def extract_pdf_text(file_path):
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text and page_text.strip():
                    text += page_text + "\n"
                else:

                    try:
                        page_image = page.to_image(resolution=300).original
                        ocr_text = pytesseract.image_to_string(page_image)
                        text += ocr_text + "\n"
                    except Exception as e:
                        print("PDF OCR error on page:", e)
    except Exception as e:
        print("PDF extraction error:", e)
    return clean_text(text)
def extract_docx_text(file_path):
    text = ""
    try:
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            if para.text.strip():
                text += para.text + "\n"
    except Exception as e:
        print("DOCX extraction error:", e)
    return clean_text(text)
def extract_pptx_text(file_path):
    text = ""
    try:
        prs = Presentation(file_path)
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    text += shape.text + "\n"
    except Exception as e:
        print("PPTX extraction error:", e)
    return clean_text(text)
def extract_xlsx_text(file_path):
    text = ""
    try:
        wb = openpyxl.load_workbook(file_path, data_only=True)
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            for row in ws.iter_rows(values_only=True):
                row_text = " ".join([str(cell) for cell in row if cell is not None])
                if row_text.strip():
                    text += row_text + "\n"
    except Exception as e:
        print("XLSX extraction error:", e)
    return clean_text(text)
def extract_image_text(file_path):
    text = ""
    try:
        img = Image.open(file_path)
        img = img.convert("L")
        text = pytesseract.image_to_string(img)
    except Exception as e:
        print("Image OCR error:", e)
    return clean_text(text)
def clean_text(text):
    if not text:
        return ""

    text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])

    text = re.sub(r"[^\x20-\x7E\n]", " ", text)
    return text.strip()
def extract_text_from_file(file_path):
    ftype = detect_file_type(file_path)
    if ftype == "pdf":
        return extract_pdf_text(file_path), "PDF"
    elif ftype == "docx":
        return extract_docx_text(file_path), "DOCX"
    elif ftype == "pptx":
        return extract_pptx_text(file_path), "PPTX"
    elif ftype == "xlsx":
        return extract_xlsx_text(file_path), "XLSX"
    elif ftype == "image":
        return extract_image_text(file_path), "IMAGE"
    else:
        try:
            with open(file_path, "r", errors="ignore") as f:
                return clean_text(f.read()), "TEXT"
        except:
            return "", "UNKNOWN"


In [ ]:

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM
import torch
ner_model_name = "dslim/bert-base-NER"
ner = pipeline(
    "ner",
    model=ner_model_name,
    grouped_entities=True,
    device=0 if torch.cuda.is_available() else -1
)
summ_model_name = "facebook/bart-large-cnn"
summarizer = pipeline(
    "summarization",
    model=summ_model_name,
    device=0 if torch.cuda.is_available() else -1
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:

import re

REGEX_PATTERNS = {
    "EMAIL": re.compile(r"\b[a-zA-Z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"),
    "IP_ADDRESS": re.compile(r"\b(?:\d{1,3}\.){3}\d{1,3}\b"),
    "PHONE": re.compile(r"\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{3,4}\b"),
    "DATE": re.compile(r"\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}-\d{2}-\d{2})\b"),

}

def mask_structured_pii(text):
    found = set()
    out = text
    for label, pat in REGEX_PATTERNS.items():
        for m in pat.finditer(out):
            snippet = m.group(0)
            found.add(label)
            out = out.replace(snippet, f"[REDACTED_{label}]")
    return out, list(found)

def mask_ner_pii(text):
    """
    Use HF NER pipeline output (grouped_entities=True)
    The pipeline returns entities with start/end offsets when available - we'll replace using offsets
    """

    if not text or not text.strip():
        return text, []

    try:
        ents = ner(text)
    except Exception as e:
        print("NER error:", e)
        ents = []

    if not ents:
        return text, []
    spans = []
    for ent in ents:
        if 'start' in ent and 'end' in ent:
            spans.append((ent['start'], ent['end'], ent['entity_group']))
        else:
            spans.append((None, None, ent['entity_group'], ent['word']))
    if all(s[0] is not None for s in spans):
        masked = []
        last_idx = 0
        out_chars = list(text)
        for start, end, label in sorted(spans, key=lambda x: x[0], reverse=True):
            out_chars[start:end] = list(f"[REDACTED_{label}]")
        out_text = "".join(out_chars)
        labels_found = list({lab for _,_,lab in spans})
        return out_text, labels_found
    else:
        out = text
        labels_found = []
        for ent in ents:
            w = ent.get('word', None)
            lab = ent.get('entity_group', None)
            if w and lab:
                token = w.replace("##","").strip()
                if token:
                    if token in out:
                        out = out.replace(token, f"[REDACTED_{lab}]")
                        labels_found.append(lab)
        return out, list(set(labels_found))

In [ ]:
def extract_security_insights(text):
    findings = []
    fw_pattern = re.compile(r"\b(ALLOW|DENY)\b.*?(?:\b(?:\d{1,3}\.){3}\d{1,3}\b(?::\d+)?|\bport\s+\d+\b|\bport:\s*\d+\b)", flags=re.IGNORECASE)
    for m in fw_pattern.finditer(text):
        findings.append(("FIREWALL_RULE", m.group(0).strip()))
    iam_pattern = re.compile(r'("Effect"\s*:\s*"(Allow|Deny)"|IAM\s+policy|"Action"\s*:)', flags=re.IGNORECASE)
    for m in iam_pattern.finditer(text):
        start = max(0, m.start()-50)
        end = min(len(text), m.end()+150)
        findings.append(("IAM_POLICY", text[start:end].strip()))
    ip_pattern = REGEX_PATTERNS["IP_ADDRESS"]
    ips = sorted(set(ip_pattern.findall(text)))
    for ip in ips:
        findings.append(("IP", ip))
    port_pattern = re.compile(r"\bport[:\s]*\d{1,5}\b", flags=re.IGNORECASE)
    for m in port_pattern.finditer(text):
        findings.append(("PORT", m.group(0)))
    grouped = {}
    for t, v in findings:
        grouped.setdefault(t, []).append(v)
    return grouped


In [ ]:
def summarize_text(text, chunk_size=800, max_length=150, min_length=40):
    """
    Summarize long text by chunking it into smaller parts.
    """
    if not text.strip():
        return "No content to summarize"

    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)

    summaries = []
    for i, chunk in enumerate(chunks):
        try:
            summary = summarizer(
                chunk,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )[0]['summary_text']
            summaries.append(summary)
        except Exception as e:
            print(f"Summarizer error on chunk {i}:", e)
            summaries.append(chunk[:300])
    combined_summary = " ".join(summaries)
    if len(combined_summary.split()) > chunk_size:
        try:
            final_summary = summarizer(
                combined_summary,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )[0]['summary_text']
            return final_summary
        except:
            return combined_summary[:1000]

    return combined_summary


In [ ]:


def process_file(file_path):
    raw_text, file_type = extract_text_from_file(file_path)
    if not raw_text.strip():
        raw_text = ""


    ner_masked_text, ner_labels = mask_ner_pii(raw_text)


    final_text, structured_labels = mask_structured_pii(ner_masked_text)

    pii_removed = list(set(ner_labels + structured_labels))

    summary = summarize_text(final_text)
    insights = extract_security_insights(final_text)
    key_findings = []
    for k, vals in insights.items():
        for v in vals[:5]:
            key_findings.append(f"{k}: {v}")
    record = {
        "file_name": os.path.basename(file_path),
        "file_type": file_type,
        "summary": summary,
        "pii_removed": ", ".join(pii_removed) if pii_removed else "",
        "key_findings": "; ".join(key_findings) if key_findings else ""
    }


    return record, final_text, insights


In [ ]:
import os
from google.colab import files
import openpyxl
import pdfplumber
import pytesseract
import pandas as pd

uploaded = files.upload()
filename = list(uploaded.keys())[0]
print("Processing:", filename)

record, cleaned_text, insights = process_file(filename)


df = pd.DataFrame([record])
df

Saving File_010.png to File_010.png
Processing: File_010.png


,file_name,file_type,summary,pii_removed,key_findings
0,File_010.png,IMAGE,Permits incoming connections to [REDACTED_MISC...,"IP_ADDRESS, MISC, ORG",


In [ ]:
import json
from google.colab import files
from google.colab import _message

nb = _message.blocking_request("get_ipynb")["ipynb"]

# Remove widget metadata
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save cleaned version
with open("cleaned_notebook.ipynb", "w") as f:
    json.dump(nb, f)

files.download("cleaned_notebook.ipynb")
